In [24]:
import json
import logging
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from train_models import create_avg_dataset
from collections import OrderedDict
from os import listdir
from os.path import isfile, join
from sklearn.externals import joblib
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
%matplotlib inline

In [2]:
BASE_PATH_DATA = '/media/ivan/DDE/datasets_proyecto_mt'
train_data_csv_paths = ['/datasets/en_train.csv', './datasets/es_train.csv', './datasets/ar_train.csv']
train_labels_json_paths = ['./authors_labels/en_train_labels.json', './authors_labels/es_train_labels.json','./authors_labels/ar_train_labels.json']
X_dataframes = [None for i in range(len(train_data_csv_paths))]
y_dataframes = [None for i in range(len(train_labels_json_paths))]
list_of_ids = [None for i in range(len(X_dataframes))]
authors_ids = []
i = 0
t_start = time()
for path_x, path_y in zip(train_data_csv_paths, train_labels_json_paths):
    list_of_ids[i] = path_x.strip().split('/')[2][:2]
    X_dataframes[i] = pd.read_csv(BASE_PATH_DATA + path_x, sep='\s*,\s*', header=0, encoding='ascii', engine='python')
#     X, y_dataframes[i] = create_avg_dataset(X, path_y)
#     y_pred_dict = OrderedDict([("class", y_dataframes[i])])
#     X_dataframes[i] = pd.concat([X.reset_index(drop=True), pd.DataFrame.from_dict(y_pred_dict)], axis=1)
    i += 1
print("{} seconds to read csv's and create all dataframes".format(time() - t_start))

77.46050834655762 seconds to read csv's and create all dataframes


In [25]:
labels = ["en", "es", "ar"]
# interesting_labels = set(["miniskirt", "velvet", "hair_spray", "maillot.1", "wig", "bikini", "stole", "brassiere", "scoreboard", "feather_boa", "bath_towel", "airliner", "silky_terrier", "bonnet", "bulletproof_vest", "maillot", "Model_T", "sarong", "overskirt", "shower_curtain", "hoopskirt", "lipstick", "plate_rack", "hand_blower", "Lhasa", "wig", "velvet", "bath_towel", "lipstick", "feather_boa", "bonnet", "face_powder", "stole", "tub", "milk_can", "bathtub", "overskirt", "golden_retriever", "Angora", "birdhouse", "fur_coat", "bikini", "Maltese_dog", "brassiere", "hair_spray", "lawn_mower", "ice_lolly", "maillot.1", "hair_slide", "chain_saw", "ballplayer", "velvet", "cradle", "brassiere", "broccoli", "lynx", "red-backed_sandpiper", "chiffonier", "Border_terrier", "lipstick", "miniskirt", "African_chameleon", "bath_towel", "safety_pin", "sea_urchin", "vase", "lighter", "football_helmet", "snorkel", "coyote", "ant", "pot", "shield", "military_uniform", "paddle"])
# interesting_labels = set(["brassiere", "velvet", "bath_towel", "lipstick"])
interesting_labels = set(["ballplayer"])
with open("authors_candidates_for_classification_only_ball.txt", "w") as file:
    threshold = 0.80
    for i in range(len(X_dataframes)):
        file.write(labels[i] + "\n")
        for index, row in X_dataframes[i].iterrows():
            for label in interesting_labels:
                if row[label] > threshold:
                    file.write("author_id = {} image_idx = {} class = {}\n".format(row['author_id'], index,row['class']))
                    file.write("{} = {} \n".format(label, row[label]))
                    file.write("{}\n".format(decode_predictions(np.array([row.drop(["author_id", "class"])]), top=3)[0]))
            

In [3]:
MODEL = VGG16(weights='imagenet')

In [29]:
def evaluate_image_representation(image_representation):
    models_dir = './models/'
    models_files_paths = [join(models_dir, f) for f in listdir(models_dir) if isfile(join(models_dir, f))]
    loaded_models = [joblib.load(model_path) for model_path in models_files_paths]
    predicted_gender = []
    genders = {"female" : 0, "male" : 0}
    model_male_predictor = []
    model_female_predictor = []
    for model_idx in range(len(loaded_models)):
        models_split = models_files_paths[model_idx].replace(models_dir, "") 
        models_split = models_files_paths[model_idx].strip().split('_')
        ids_models = []
        for s in models_split:
            if s == "svm":
                break
            ids_models.append(s)
        ids_models[0] = ids_models[0][-2:]
#         print("Using trained model {}".format(ids_models))
        y = loaded_models[model_idx].predict(image_representation)[0]
        genders[y] += 1
        if y == "female":
            model_female_predictor.append("-".join(ids_models))
    if genders["male"] == 7:
        print("Todos los modelos predijeron:\nhombre")
    else:
        print("Excepto: {}".format(", ".join(model_female_predictor)))

In [35]:
img_dir = '/home/ivan/Escritorio/ballplayer/es'
img_files = [join(img_dir, f) for f in listdir(img_dir) if isfile(join(img_dir, f))]
for image_path in img_files:
    print(image_path)
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    features = MODEL.predict(image)
    decoded = [t[1] for t in decode_predictions(features, top=3)[0]]
    print('Atributos (top 3): \n* {}'.format("\n* ".join(decoded)))
    evaluate_image_representation(features)

/home/ivan/Escritorio/ballplayer/es/242e15635ee82ffcfca39e31ccf0165d.0.jpeg
Atributos (top 3): 
* soccer_ball
* football_helmet
* parachute
Todos los modelos predijeron:
hombre
/home/ivan/Escritorio/ballplayer/es/999ce7ac3ae2b7ee747e68158e96d70d.1.jpeg
Atributos (top 3): 
* ballplayer
* baseball
* football_helmet
Todos los modelos predijeron:
hombre
/home/ivan/Escritorio/ballplayer/es/b3560fad2e4486269f56e012b2bf3918.8.jpeg
Atributos (top 3): 
* ballplayer
* baseball
* football_helmet
Todos los modelos predijeron:
hombre
